# 機械学習の自動化

このノートブックでは、MNISTの数字データセットを用いて、Auto MLを紹介します。今回は、Auto MLのパッケージとして、以下の2つを使います。

- TPOT
- Auto Keras

## TPOTを用いたAuto ML

TPOTは、遺伝的プログラミングを用いて機械学習パイプラインを最適化するためのツールです。以下の図の灰色で囲まれた部分のプロセスである特徴選択やモデル選択を自動化する機能を備えています。

![](https://github.com/EpistasisLab/tpot/raw/master/images/tpot-ml-pipeline.png)

### パッケージのインストール

In [1]:
!pip install -q scikit-learn==0.23.2 tpot==0.11.7

     |████████████████████████████████| 6.8 MB 5.4 MB/s 
     |████████████████████████████████| 87 kB 3.2 MB/s 
     |████████████████████████████████| 166.7 MB 10 kB/s 
     |████████████████████████████████| 160 kB 43.7 MB/s 


### インポート

In [2]:
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

### データセットの読み込み

今回は、データセットとして、手書き数字の画像データセットであるMNISTを利用します。したがって、画像の分類問題を解くことになります。

In [3]:
digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(
    digits.data,
    digits.target,
    train_size=0.75,
    test_size=0.25
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1347, 64), (450, 64), (1347,), (450,))

### 学習

MNISTでは分類問題を扱うので、`TPOTClassifier`を用意します。回帰問題を扱う場合は、`TPOTRegressor`を使います。

`population_size`は、世代ごとに遺伝的プログラミング集団に保持する個体数を表しています。基本的に、大きいほうが性能は高くなります。また、`max_time_mins`で学習の最大時間を設定しています。これら以外の引数については、以下のドキュメントを御覧ください。

- [TPOT API](http://epistasislab.github.io/tpot/api/)

In [4]:
tpot = TPOTClassifier(
    verbosity=2,
    max_time_mins=10,
    population_size=40,
    random_state=2021
)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

Optimization Progress:   0%|          | 0/40 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9829299187663499

10.65 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: KNeighborsClassifier(input_matrix, n_neighbors=1, p=1, weights=distance)
0.9777777777777777


ベストなパイプラインと正解率が表示されています。

## Auto-Kerasを用いたAuto ML

### パッケージのインストール

In [5]:
!pip install autokeras==1.0.16 tensorflow==2.4.0

     |████████████████████████████████| 166 kB 5.2 MB/s 
     |████████████████████████████████| 394.7 MB 17 kB/s 
     |████████████████████████████████| 97 kB 7.2 MB/s 
     |████████████████████████████████| 462 kB 48.1 MB/s 
     |████████████████████████████████| 3.8 MB 37.6 MB/s 
     |████████████████████████████████| 2.9 MB 35.0 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.40.0
    Uninstalling grpcio-1.40.0:
      Successfully uninstalled grpcio-1.40.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  At

### インポート

In [6]:
import warnings

import autokeras as ak
from tensorflow.keras.datasets import mnist

warnings.filterwarnings("ignore")

### 学習データの読み込み

In [7]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)
print(y_train[:3])

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
[5 0 4]


### モデルの学習

`ImageClassifier`で画像の分類器を初期化しています。また、`max_trials`で試すモデルの数を指定しています。時間がかかるので、今回は2にしています。

In [8]:
clf = ak.ImageClassifier(max_trials=2)
clf.fit(x_train, y_train, epochs=3)

Trial 2 Complete [00h 55m 14s]
val_loss: 0.36572402715682983

Best val_loss So Far: 0.045847151428461075
Total elapsed time: 00h 56m 35s
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
1875/1875 [==============================] - 19s 10ms/step - loss: 0.3044 - accuracy: 0.9076
Epoch 2/3
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0783 - accuracy: 0.9765
Epoch 3/3
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0619 - accuracy: 0.9810
INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


In [9]:
# ラベルの予測
predicted_y = clf.predict(x_test)
print(predicted_y)

313/313 [==============================] - 1s 3ms/step
[['7']
 ['2']
 ['1']
 ...
 ['4']
 ['5']
 ['6']]


In [11]:
# 評価
_, acc = clf.evaluate(x_test, y_test)
acc

313/313 [==============================] - 2s 6ms/step - loss: 0.0438 - accuracy: 0.9848


0.9847999811172485